# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 43 new papers today
          15 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/14 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.00669


extracting tarball to tmp_2209.00669...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.00759


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2209.00669/natbib.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.01166


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'main.bbl' from 'tmp_2209.01145/main.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2209.01166...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'main.bbl' from 'tmp_2209.01166/main.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00828-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00828) | **Triage of the Gaia astrometric orbits. I. A sample of binaries with  probable compact companions**  |
|| Sahar Shahaf, et al. -- incl., <mark>Hans-Walter Rix</mark> |
|*Appeared on*| *2022-09-05*|
|*Comments*| *Submitted to MNRAS; 12 pages, 13 figures*|
|**Abstract**| In preparation for the release of the astrometric orbits of Gaia, Shahaf et al. (2019) proposed a triage technique to identify astrometric binaries with compact companions based on their astrometric semi-major axis, parallax, and primary mass. The technique requires the knowledge of the appropriate mass-luminosity relation to rule out single or close-binary main-sequence companions. The recent publication of the Gaia DR3 astrometric orbits used a schematic version of this approach, identifying 735 astrometric binaries that might have compact companions. In this communication, we return to the triage of the DR3 astrometric binaries with more careful analysis, estimating the probability for its astrometric secondary to be a compact object or a main-sequence close binary. We compile a sample of 177 systems with highly-probable non-luminous massive companions, which is smaller but cleaner than the sample reported in Gaia DR3. The new sample includes 8 candidates to be black-hole systems with compact-object masses larger than 2.4 $M_\odot$. The orbital-eccentricity$-$secondary-mass diagram of the other 169 systems suggests a tentative separation between the white-dwarf and the neutron-star binaries. Most white-dwarf binaries are characterized by small eccentricities of about 0.1 and masses of 0.6 $M_\odot$, while the neutron star binaries display typical eccentricities of 0.4 and masses of 1.3 $M_\odot$. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.01125-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.01125) | **RVSPY -- Radial Velocity Survey for Planets around Young Stars. Target  characterization and high-cadence survey**  |
|| <mark>O. Zakhozhay</mark>, et al. -- incl., <mark>R. Launhardt</mark>, <mark>A. Mueller</mark>, <mark>A. Hempel</mark>, <mark>M. Hempel</mark>, <mark>Th. Henning</mark>, <mark>M. Kuerster</mark>, <mark>A. Pavlov</mark>, <mark>T. Trifonov</mark> |
|*Appeared on*| *2022-09-05*|
|*Comments*| *24 pages, 14 figures, 4 tables; Accepted for publication in A&A*|
|**Abstract**| We introduce our Radial Velocity Survey for Planets around Young stars (RVSPY), characterise our target stars, and search for substellar companions at orbital separations smaller than a few au from the host star. We use the FEROS spectrograph to obtain high signal-to-noise spectra and time series of precise radial velocities (RVs) of 111 stars most of which are surrounded by debris discs. Our target stars have spectral types between early F and late K, a median age of 400 Myr, and a median distance of 45 pc. We determine for all target stars their basic stellar parameters and present the results of the high-cadence RV survey and activity characterization. We achieve a median single-measurement RV precision of 6 m/s and derive the short-term intrinsic RV scatter of our targets (median 22 m/s), which is mostly caused by stellar activity and decays with age from >100 m/s at <20 Myr to <20 m/s at >500 Myr. We discover six previously unknown close companions with orbital periods between 10 and 100 days, three of which are low-mass stars, and three are in the brown dwarf mass regime. We detect no hot companion with an orbital period <10 days down to a median mass limit of ~1 M_Jup for stars younger than 500 Myr, which is still compatible with the established occurrence rate of such companions around main-sequence stars. We find significant RV periodicities between 1.3 and 4.5 days for 14 stars, which are, however, all caused by rotational modulation due to starspots. We also analyse the TESS photometric time series data and find significant periodicities for most of the stars. For 11 stars, the photometric periods are also clearly detected in the RV data. We also derive stellar rotation periods ranging from 1 to 10 days for 91 stars, mostly from TESS data. From the intrinsic activity-related short-term RV jitter, we derive the expected mass-detection thresholds for longer-period companions. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.01145-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.01145) | **Optimisation of the WEAVE target assignment algorithm**  |
|| <mark>Sarah Hughes</mark>, et al. -- incl., <mark>Daniel Smith</mark>, <mark>Ian Lewis</mark> |
|*Appeared on*| *2022-09-05*|
|*Comments*| *19 pages, 17 figures*|
|**Abstract**| WEAVE is the new wide-field spectroscopic facility for the prime focus of the William Herschel Telescope in La Palma, Spain. Its fibre positioner is essential for the accurate placement of the spectrograph's ~960-fibre multiplex. To maximise the assignment of its optical fibres, WEAVE uses a simulated annealing algorithm called Configure, which allocates the fibres to targets in the field of view. We have conducted an analysis of the algorithm's behaviour using a subset of mid-tier WEAVE-LOFAR fields, and adjusted the priority assignment algorithm to optimise the total fibres assigned per field, and the assignment of fibres to the higher priority science targets. The output distributions have been examined, to investigate the implications for the WEAVE science teams. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00856-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00856) | **Characterization of the MASCOT landing area by Hayabusa2**  |
|| Stefan Schröder, et al. -- incl., <mark>Frank Scholten</mark> |
|*Appeared on*| *2022-09-05*|
|*Comments*| **|
|**Abstract**| Context. After landing on C-type asteroid Ryugu, MASCOT imaged brightly colored, submillimeter-sized inclusions in a small rock. Hayabusa2 successfully returned a sample of small particles from the surface of Ryugu, but none of these appear to harbor such inclusions. The samples are considered representative of Ryugu. Aims. To understand the apparent discrepancy between MASCOT observations and Ryugu samples, we assess whether the MASCOT landing site, and the rock by implication, is perhaps atypical for Ryugu. Methods. We analyzed observations of the MASCOT landing area acquired by three instruments on board Hayabusa2: a camera (ONC), a near-infrared spectrometer (NIRS3), and a thermal infrared imager (TIR). We compared the landing area properties thus retrieved with those of the average Ryugu surface. Results. We selected several areas and landforms in the landing area for analysis: a small crater, a collection of smooth rocks, and the landing site itself. The crater is relatively blue and the rocks are relatively red. The spectral and thermophysical properties of the landing site are very close to those of the average Ryugu surface. The spectral properties of the MASCOT rock are probably close to average, but its thermal inertia may be somewhat higher. Conclusions. The MASCOT rock can also be considered representative of Ryugu. Some of the submillimeter-sized particles in the returned samples stand out because of their atypical spectral properties. Such particles may be present as inclusions in the MASCOT rock. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00902-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00902) | **KMTNet Nearby Galaxy Survey: Overview and a Survey Description**  |
|| Woowon Byun, et al. -- incl., <mark>Joon Hyeop Lee</mark>, <mark>Yongseok Lee</mark> |
|*Appeared on*| *2022-09-05*|
|*Comments*| *32 pages, 21 figures, accepted for publication in PASP*|
|**Abstract**| Recently, there has been an increasing demand for deep imaging surveys to investigate the history of the mass assembly of galaxies in detail by examining the remnants of mergers and accretions, both of which have very low surface brightness (LSB). In addition, the nature of star formation in LSB regions, such as galaxy outer disks, is also an intriguing topic in terms of understanding the physical mechanisms of disk evolution. To address these issues, this study conducts a survey project, called the Korea Microlensing Telescope Network (KMTNet) Nearby Galaxy Survey to construct a deep imaging data set of nearby galaxies in the southern hemisphere using KMTNet. It provides deep and wide-field images with a field-of-view of $\sim$12 deg$^2$ for 13 nearby galaxies drawn from the Carnegie-Irvine Galaxy Survey catalog, in optical broadbands ($BRI$) and an H$\alpha$ narrowband. Through a dedicated data reduction, the surface brightness limit in 10$^{\prime\prime}\times10^{\prime\prime}$ boxes was found to reach as deep as $\mu_{1\sigma}\sim29$-31 mag arcsec$^{-2}$ in the optical broadbands and $f_{1\sigma}\sim1$-$2\times 10^{-18}$ erg s$^{-1}$ cm$^{-2}$ arcsec$^{-2}$ in the H$\alpha$ narrowband. To conclude the paper, several possible scientific applications for this data set are described. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00970-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00970) | **An intermediate polar candidate toward the Galactic plane**  |
|| Samaresh Mondal, et al. -- incl., <mark>Frank Haberl</mark> |
|*Appeared on*| *2022-09-05*|
|*Comments*| *6 pages, 5 figures, accepted for publication in A&A*|
|**Abstract**| For the past decade, it has been suggested that intermediate polars (IPs), a subclass of magnetic cataclysmic variables (CVs), are one of the main contributors to the hard diffuse X-ray emission from the Galactic center (GC) and Galactic ridge. In our ongoing \emph{XMM-Newton} survey of the central region of the Galactic disk ($20^\circ\times2^\circ$), we detected a persistent IP candidate, $1.7^\circ$ away from the GC. In this work, we better characterize the behavior of this source by looking at the new and archival XMM-Newton data. We performed a detailed X-ray spectral modeling of the source. Furthermore, we searched for X-ray pulsations in the light curve as well as its counterpart at other wavelengths. The XMM-Newton spectrum (0.8--10 keV) of the source is described by a partial covering collisionally ionized diffuse gas with plasma temperature $kT=15.7^{+20.9}_{-3.6}$ keV. In addition, the spectrum shows the presence of iron lines at $E=6.44$, 6.65, and 6.92 keV with equivalent widths of $194^{+89}_{-70}$, $115^{+79}_{-75}$, and $98^{+93}_{-74}$ eV, respectively. The X-ray light curve shows a coherent modulation with a period of $P=432.44\pm0.36$ s, which we infer is the spin period of the white dwarf. The white dwarf mass estimated from fitting a physical model to the spectrum results in $M_{\rm WD}=1.05^{+0.16}_{-0.21}\ M_{\odot}$. We were able to find a likely optical counterpart in the Gaia catalog with a G magnitude of 19.26, and the distance to the source derived from the measured Gaia parallax is $\sim$4.3 kpc. We provide an improved source localization with subarcsec accuracy. The spectral modeling of the source indicates the presence of intervening circumstellar gas, which absorbs the soft X-ray photons. The measured equivalent width of the iron lines and the detection of the spin period in the light curve are consistent with those from IPs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00971-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00971) | **Photometric and dynamic characterisation of active asteroid (248370)  2005 QN173**  |
|| Bojan Novakovic, Debora Pavela, <mark>Henry H. Hsieh</mark>, Dusan Marceta |
|*Appeared on*| *2022-09-05*|
|*Comments*| *Accepted for publication in MNRAS*|
|**Abstract**| We present the physical and dynamical properties of the recently discovered active asteroid (248370) 2005 QN173 (aka 433P). From our observations, we derived two possible rotation period solutions of 2.7 and 4.1 hours. The corresponding light curve amplitudes computed after correcting for the effect of coma are 0.28 and 0.58 mag, respectively. Both period solutions are shorter than the critical rotation limit computed for a strengthless triaxial ellipsoid, suggesting that rotation mass shedding should at least partly be responsible for the observed activity. We confirm that the activity level is fading further, but at a very modest rate of only 0.006 mag/day, still also compatible with sublimation-driven activity. We found that 248370 likely belongs to the Themis asteroid family, making it a fourth main-belt comet associated with this group. Orbital characteristics of 248370 are also consistent with its origin in the young 288P cluster of asteroids. The 288P cluster is associated with its namesake main-belt comet, providing an exciting possibility for a comparative analysis of intriguing main-belt comets 248370 and 288P. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.01040-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.01040) | **TOPz: Photometric redshifts for J-PAS**  |
|| J. Laur, et al. -- incl., <mark>A. Hernán-Caballero</mark>, <mark>J. Chaves-Montero</mark> |
|*Appeared on*| *2022-09-05*|
|*Comments*| *20 pages, 24 figures*|
|**Abstract**| The importance of photometric galaxy redshift estimation is rapidly increasing with the development of specialised powerful observational facilities. We develop a new photometric redshift estimation workflow TOPz to provide reliable and efficient redshift estimations for the upcoming large-scale survey J-PAS which will observe 8500 deg2 of the northern sky through 54 narrow-band filters. TOPz relies on template-based photo-z estimation with some added J-PAS specific features and possibilities. We present TOPz performance on data from the miniJPAS survey, a precursor to the J-PAS survey with an identical filter system. First, we generated spectral templates based on the miniJPAS sources using the synthetic galaxy spectrum generation software CIGALE. Then we applied corrections to the input photometry by minimising systematic offsets from the template flux in each filter. To assess the accuracy of the redshift estimation, we used spectroscopic redshifts from the DEEP2, DEEP3, and SDSS surveys, available for 1989 miniJPAS galaxies with r < 22 magAB. We also tested how the choice and number of input templates, photo-z priors, and photometric corrections affect the TOPz redshift accuracy. The general performance of the combination of miniJPAS data and the TOPz workflow fulfills the expectations for J-PAS redshift accuracy. Similarly to previous estimates, we find that 38.6% of galaxies with r < 22 mag reach the J-PAS redshift accuracy goal of dz/(1 + z) < 0.003. Limiting the number of spectra in the template set improves the redshift accuracy up to 5%, especially for fainter, noise-dominated sources. Further improvements will be possible once the actual J-PAS data become available. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.00828.md
    + _build/html/tmp_2209.00828/./graphics/M2e_classIII.png
    + _build/html/tmp_2209.00828/./graphics/AMRF_model_part2.png
    + _build/html/tmp_2209.00828/./graphics/MLR_model.png
    + _build/html/tmp_2209.00828/./graphics/m2_classIII_hist.png
exported in  _build/html/2209.01125.md
    + _build/html/tmp_2209.01125/./fig_RVSPY_plot_teff-rmsrv_2022Aug.png
    + _build/html/tmp_2209.01125/./fig_RVSPY_plot_sigRV-teff-vsini_2022Aug.png
    + _build/html/tmp_2209.01125/./fig_RVSPY_obs111_Teff_Mpdetlim_2022Aug.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{\var}[1]{\DTLfetch{main_data}{thekey}{#1}{thevalue}}$
$\newcommand$
$\newcommand{\qmin}{\mathcal{Q}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{$\var$}[1]{\DTLfetch{main_data}{thekey}{#1}{thevalue}}$
$\newcommand$
$\newcommand{$\qmin$}{\mathcal{Q}}$
$\newcommand{$\red$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

#  astrometric orbits. I. A sample of binaries with probable compact companions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.00828-b31b1b.svg)](https://arxiv.org/abs/2209.00828)<mark>Appeared on: 2022-09-05</mark> - _Submitted to MNRAS; 12 pages, 13 figures_

</div>
<div id="authors">

Sahar Shahaf, et al. -- incl., <mark><mark>Hans-Walter Rix</mark></mark>

</div>
<div id="abstract">

**Abstract:** In preparation for the release of the astrometric orbits of{\it Gaia},\citet[][]{shahaf19}proposed a triage technique to identify astrometric binaries with compact companions based on their astrometric semi-major axis, parallax, and primary mass. The technique requires the knowledge of the appropriate mass-luminosity relation to rule out single or close-binary main-sequence companions. The recent publication of the\textit{Gaia}DR3 astrometric orbits used a schematic version of this approach, identifying$735$astrometric binaries that might have compact companions. In this communication, we return to the triage of the DR3 astrometric binaries with more careful analysis, estimating the probability for its astrometric secondary to be a compact object or a main-sequence close binary. We compile a sample of$\var{N_class3}$systems with highly-probable non-luminous massive companions, which is smaller but cleaner than the sample reported in\textit{Gaia}DR3. The new sample includes$8$candidates to be black-hole systems with compact-object masses larger than$2.4$\solarmass. The orbital-eccentricity--secondary-mass diagram of the other$169$systems suggests a tentative separation between the white-dwarf and the neutron-star binaries. Most white-dwarf binaries are characterized by small eccentricities of about$0.1$and masses of$0.6$\solarmass, while the neutron star binaries display typical eccentricities of$0.4$and masses of$1.3$\solarmass.

</div>

<div id="div_fig1">

<img src="tmp_2209.00828/./graphics/M2e_classIII.png" alt="Fig2" width="100%"/>

**Figure 2. -** Orbital eccentricity versus secondary mass for the class-III sample.  We set the upper boundary of the mass axis at $2.1$$\solarmass$, for clarity. Lines represent the central regions of the fitted Gaussian mixture model (see text). Systems identified as members of the WD and NS clusters appear as white and blue circles, respectively. Lines follow a constant $\ln$-likelihood level of $-0.5$, based on the fitted Gaussian mixture model, and highlight the main loci of the secondary-mass--eccentricity distribution. Centres of the three Gaussian components are plotted as black "+" signs. (*fig:co_e-m*)

</div>
<div id="div_fig2">

<img src="tmp_2209.00828/./graphics/AMRF_model_part2.png" alt="Fig8.1" width="50%"/><img src="tmp_2209.00828/./graphics/MLR_model.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Maximum $\mathcal{A}$ for an MS secondary (dotted line) and an MS close-binary companion (dashed line) as a function of primary mass, based on \citet{mamajek13} observed MLR. Black circles and squares show the maximum values of Fig. \ref{fig:amrf_curve1}. Purple and light-blue stripes illustrate the expected $\mathcal{A}$ of white dwarfs, at $0.45{-}0.75 \solarmass$, and neutron stars, at $1.4{-}2.1  \solarmass$ companions, respectively. Note that their position does not depend on the period or the parallax of the system. The figure suggests that since some white-dwarf and most neutron-star companions are expected to reside above the corresponding maximum $\mathcal{A}$, they can be identified as such.Impact of stellar age and metallicity on  $\mathcal{A}_{$\textsc${tr}}$. Class-III limiting value, $\mathcal{A}_{$\textsc${tr}}$, as a function of the primary mass. Dashed red and blue lines represent the MIST-based limits for old and young stellar populations, respectively. Grey dashed line represents the limit obtained from the empirical MLR of \citet{mamajek13}(dashed), which were used in the \textit{NSS}. Grey solid line is the limiting curve adopted by this work. See Section \ref{sec: age metallicity based limits} and Appendix \ref{APP: Atr}. \newline\newline\newline (*fig:amrf_curve*)

</div>
<div id="div_fig3">

<img src="tmp_2209.00828/./graphics/m2_classIII_hist.png" alt="Fig3" width="100%"/>

**Figure 3. -** Secondary-mass histogram of the class-III sample. The most prominent peak, at ${\sim}0.6$$\solarmass$ corresponds to WD secondaries in the sample, and another, less prominent peak, is located at ${\sim}1.3$$\solarmass$. We set the upper boundary of the mass axis at $2.1$$\solarmass$, for clarity. As a result, $8$ additional systems with secondary masses larger than $2.4$$\solarmass$ are not shown. Lines represent the marginal distributions derived from a Gaussian-mixture model that was fitted to the secondary-mass--eccentricity diagram (see Section \ref{sec: m2-e}). The overlap between the two classes is small, as only one bin shares points from both classes, with two WDs and one NS.
         (*fig:hist_m2*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\amu}[1]{{\leavevmode\color{dark-blue} #1}}$
$\newcommand{\trifon}[2]{{\leavevmode\color{dark-green} #1}}$
$\newcommand{\sr}[1]{{\leavevmode\color{medium-blue} #1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\amu$}[1]{{\leavevmode\color{dark-blue} #1}}$
$\newcommand{$\trifon$}[2]{{\leavevmode\color{dark-green} #1}}$
$\newcommand{$\sr$}[1]{{\leavevmode\color{medium-blue} #1}}$</div>



<div id="title">

# RVSPY - Radial Velocity Survey for Planets around Young stars: Target characterisation and high-cadence survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.01125-b31b1b.svg)](https://arxiv.org/abs/2209.01125)<mark>Appeared on: 2022-09-05</mark> - _24 pages, 14 figures, 4 tables; Accepted for publication in A&A_

</div>
<div id="authors">

<mark><mark>O. Zakhozhay</mark></mark>, et al. -- incl., <mark><mark>R. Launhardt</mark></mark>, <mark><mark>A. Mueller</mark></mark>, <mark><mark>A. Hempel</mark></mark>, <mark><mark>M. Hempel</mark></mark>, <mark><mark>Th. Henning</mark></mark>, <mark><mark>M. Kuerster</mark></mark>, <mark><mark>A. Pavlov</mark></mark>, <mark><mark>T. Trifonov</mark></mark>

</div>
<div id="abstract">

**Abstract:** The occurrence rate and period distribution of (giant) planets around young stars is still not as well constrained as for older main-sequence stars. This is mostly due to the intrinsic activity-related complications and the avoidance of young stars in many large planet search programmes. Yet, dynamical restructuring processes in planetary systems may last significantly longer than the actual planet formation phase and may well extend long into the debris disc phase, such that the planet populations around young stars may differ from those observed around main-sequence stars.We introduce our Radial Velocity Survey for Planets around Young stars (RVSPY), which is closely related to the\mbox{NaCo-ISPY}direct imaging survey, characterise our target stars, and search for substellar companions at orbital separations smaller than a few au from the host star.We used the FEROS spectrograph, mounted to the MPG/ESO 2.2 m telescope in Chile, to obtain high signal-to-noise spectra and time series of precise radial velocities (RVs) of 111 stars, most of which are surrounded by debris discs. Our target stars have spectral types between early F and late K, a median age of 400 Myr, and a median distance of 45 pc. During the initial reconnaissance phase of our survey, we determined stellar parameters and used high-cadence observations to characterise the intrinsic stellar activity, searched for hot companions with orbital periods of up to 10 days, and derived the detection thresholds for longer-period companions. In our analysis we, have included archival spectroscopic data, spectral energy distribution, and data for photometric time series from the TESS mission.For all target stars we determined their basic stellar parameters and present the results of the high-cadence RV survey and activity characterisation. We have achieved a median single-measurement RV precision of 6 m/s and derived the short-term intrinsic RV scatter of our targets (median 23 m/s), which is mostly caused by stellar activity and decays with an age from >100 m/s at <20 Myr to <20 m/s at >500 Myr.We analysed time series periodograms of the high-cadence RV data and the shape of the individual cross-correlation functions. We discovered six previously unknown close companions with orbital periods between 10 and 100 days, three of which are low-mass stars, and three are in the brown dwarf mass regime. We detected no hot companion with an orbital period <10 days down to a median mass limit of$\sim$1 M$_{\rm Jup}$for stars younger than 500 Myr, which is still compatible with the established occurrence rate of such companions around main-sequence stars.We found significant RV periodicities between 1.3 and 4.5 days for 14 stars, which are, however, all caused by rotational modulation due to starspots. We also analysed the data for TESS photometric time series and found significant periodicities for most of the stars. For 11 stars, the photometric periods are also clearly detected in the RV data.We also derived stellar rotation periods ranging from 1 to 10 days for 91 stars, mostly from the TESS data.From the intrinsic activity-related short-term RV jitter, we derived the expected mass-detection thresholds for longer-period companions, and selected 84 targets for the longer-term RV monitoring.\looseness=-4

</div>

<div id="div_fig1">

<img src="tmp_2209.01125/./fig_RVSPY_plot_teff-rmsrv_2022Aug.png" alt="Fig6" width="100%"/>

**Figure 6. -** \label{fig:rmsrv_teff}
Intrinsic rms scatter of the RVs in the 2-week high-cadence data vs. (spectroscopic) effective stellar temperature, $T_{\rm eff}$, with age encoded in colour. Stars marked as triangles do not have significant IR excess (see Sect. \ref{ssec:targets:sel}).
The horizontal dashed-dotted line marks the approximate boundary above which we consider RV exoplanet searches not feasible (although we use a mass detection threshold in the end; see Sect. \ref{sec:dis:prospects} and Fig. \ref{fig:Mp_vs_Teff}).
 (*fig:rmsrv_teff*)

</div>
<div id="div_fig2">

<img src="tmp_2209.01125/./fig_RVSPY_plot_sigRV-teff-vsini_2022Aug.png" alt="Fig8" width="100%"/>

**Figure 8. -** \label{fig:vsini}
Relation between single-measurement RV precision, $\sigma_{RV}$, and $T_{\rm eff}$({\it left}) and $v\sin(i)$({\it right}). Stellar ages are colour-coded. Stars marked as triangles do not have significant IR excess (see Sect. \ref{ssec:targets:sel}).
Horizontal dashed-dotted lines mark the anticipated threshold value for the RV precision of $\sigma_{RV}=50$ m/s for our longer-term survey (Sect. \ref{ssec:obs:strat}). Vertical dashed-dotted lines indicate the values of $T_{\rm eff}=6400$ K (left panel) and $v\sin(i)=40$ km/s (right panel), above which many target are no longer compatible with our $\sigma_{RV}\leq50$ m/s single-measurement RV-precision goal. The vertical dotted line on the right panel marks the lower sensitivity limit (3 km/s) of our $v\sin(i)$ measurements (Sect. \ref{ssec:res:par}). The light-blue dashed lines in the right panel show the best linear fits to the correlation between $v\sin(i)$ and $\log(\sigma_{RV})$ for $v\sin(i)\leq40$ km/s and $v\sin(i)\geq40$ km/s.
 (*fig:vsini*)

</div>
<div id="div_fig3">

<img src="tmp_2209.01125/./fig_RVSPY_obs111_Teff_Mpdetlim_2022Aug.png" alt="Fig11" width="100%"/>

**Figure 11. -** \label{fig:Mp_vs_Teff}
Mass detection limits for hypothetical companions with $P=1$ yr that induce an RV amplitude three times larger than the activity-jitter rms derived from the high-cadence observations, plotted vs. host star (spectroscopic) $T_{\rm eff}$. Ages are coded in colour. Stars marked as triangles do not have a significant IR excess (see Sect. \ref{ssec:targets:sel}).
The horizontal dashed-dotted line marks the approximate boundary between the low-mass stellar and substellar regimes and the dashed line marks the approximate boundary between the planetary and brown-dwarf mass regimes. The dotted line marks 3 M$_{\rm Jup}$.
 (*fig:Mp_vs_Teff*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

24  publications in the last 7 days.
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers